In [3]:
import os
import re
import gensim
import pickle
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer

In [4]:
output_dir = 'output4'
stop_words_path = './dataset/english'

In [5]:
def remove_stop_words(inputs, stop_words_path):
    '''
    去除停用词
    :param inputs: [word1 word2...]
    :param stop_words_path:
    :return:
    '''
    with open(stop_words_path, "r", encoding="utf-8") as fr:
        stop_words = [line.strip() for line in fr.readlines()]
    outputs = [word for word in inputs if word not in stop_words]
    return outputs

In [6]:
def stem_words(inputs):
    '''
    词干化处理
    :param inputs: [word1 word2...]
    :return:
    '''
    stemmer = SnowballStemmer('english')
    # stemmer = PorterStemmer()
    outputs = [stemmer.stem(word) for word in inputs]
    return outputs

In [7]:
def pad_forward(in_file, out_file, summary_len, description_len):
    '''对词向量进行截取或者填充'''
    print("pad_forward", in_file)
    nn_data = []
    for data in pickle.load(open(in_file, 'rb')):
        summary_sent = data[1]
        description_sent = data[2]
        if len(summary_sent) >= summary_len:
            summary_sent = summary_sent[:summary_len]
        else:
            pad = [0] * (summary_len - len(summary_sent))
            summary_sent = pad + summary_sent

        if len(description_sent) >= description_len:
            description_sent = description_sent[:description_len]
        else:
            pad = [0] * (description_len - len(description_sent))
            description_sent = pad + description_sent
        nn_data.append([data[0], summary_sent, description_sent, data[3], data[4]])
    pickle.dump(nn_data, open(out_file, 'wb'))

In [8]:
def preprocess_project(project_name):

    project_dir = output_dir + '\\' + project_name
    if not os.path.exists(project_dir):
        os.makedirs(project_dir)

    #读取pkl文件
    input_file = open("dataset/" + project_name + ".pkl", "rb")
    info = pickle.load(input_file)
    input_file.close()

    #处理数据
    key = []
    key_id = []
    summary = []
    description = []
    priority = []
    for i in range(len(info)):
        key.append(info[i]['key']) #'HTTPCLIENT-569'
        cur_index = info[i]['key'].find("-") #10
        key_id.append(int(info[i]['key'][cur_index + 1:])) #569
        #'HttpState.clearCookies() should be synchronized'
        summary.append(info[i]['fields']['summary'])
        """
        ('The HttpState class has a clearCookies method that is not synchronized but\n'
         'should be considering it modifies an ArrayList (which is unsynchronized). '
         'All\n'
         'other methods which modify or read from the ArrayList are synchronized '
         'except\n'
         'the clearCookies method. \n'
         '\n'
         'I stumbled upon this fact because a webapp I am working on that uses '
         'HttpClient\n'
         'threw an IllegalArgumentException indicating that one of the cookies in the\n'
         "array returned from HttpState.getCookies() was null, which shouldn't be\n"
         'possible.  Upon further inspection and testing, the only possible option is '
         'that\n'
         'the threadsafety hole left by the unsynchronized clearCookies method caused '
         'the\n'
         'issue.')
        """
        description.append(info[i]['fields']['description'])
        priority.append(info[i]['fields']['priority']['id'])
    key = np.array(key)
    key_id = np.array(key_id)
    summary = np.array(summary)
    description = np.array(description)
    priority = np.array(priority)

    indicies = np.argsort(key_id) #元素从小到大排列后提取索引
    key = key[indicies]
    summary = summary[indicies]
    description = description[indicies]
    priority = priority[indicies]

    #将summary、description分别分词、去停用词
    summary_tokenize = []
    description_tokenize = []
    for i in range(len(summary)):
        cur_summary = summary[i]
        cur_description = description[i]
        summary_words = list(word_tokenize(cur_summary))
        summary_words = [word.lower() for word in summary_words]
        summary_words = remove_stop_words(summary_words, stop_words_path)
        summary_words = stem_words(summary_words)
        try:
            description_words = list(word_tokenize(cur_description))
            description_words = [word.lower() for word in description_words]
            description_words = remove_stop_words(description_words, stop_words_path)
            description_words = stem_words(description_words)
        except Exception:
            description_words = []
        summary_tokenize.append(summary_words)
        description_tokenize.append(description_words)
    summary_tokenize = np.array(summary_tokenize)
    description_tokenize = np.array(description_tokenize)

    #将经过分词、去停用词后的summary、description中的数字和“.”置换为“<NUM>” here error
    pattern1 = re.compile("^[0-9\.]+$")
    summary_processed = []
    for i in range(len(summary_tokenize)):
        cur_summary_tokenize = []
        for word in summary_tokenize[i]:
            if pattern1.match(word) is not None:
                cur_summary_tokenize.append("<NUM>")
                continue
            cur_summary_tokenize.append(word)
        summary_processed.append(cur_summary_tokenize)

    description_processed = []
    for i in range(len(description_tokenize)):
        cur_description_tokenize = []
        for word in description_tokenize[i]:
            if pattern1.match(word) is not None:
                cur_description_tokenize.append("<NUM>")
                continue
            cur_description_tokenize.append(word)
        description_processed.append(cur_description_tokenize)

    summary_processed = np.array(summary_processed)
    description_processed = np.array(description_processed)


    #读取label并整理 classified为bug的值为1，否则为0
    info = pd.read_csv("dataset/" + project_name + "_classification_vs_type.csv")
    label = list((info['CLASSIFIED'] == "BUG").astype(int))
    label = np.array(label)
    label = label[indicies]

    word2index = {} #word: index
    index_label = [] #[key,[summary_index...],[description_index...],label]
    index = 1  # 0 used for padding
    for i in range(len(summary_processed)):
        summary_index = []
        description_index = []

        for word in summary_processed[i]:
            if word not in word2index:
                word2index[word] = index
                summary_index.append(index)
                index += 1
            else:
                summary_index.append(word2index[word])

        for word in description_processed[i]:
            if word not in word2index:
                word2index[word] = index
                description_index.append(index)
                index += 1
            else:
                description_index.append(word2index[word])

        index_label.append([key[i], summary_index, description_index, priority[i], label[i]])

    #将word2index、index_label写入文件
    pickle.dump(word2index, open(project_dir + "/word2index.pkl", 'wb'))
    pickle.dump(index_label, open(project_dir + "/index_label.pkl", 'wb'))

    # 对句向量进行截取或填充
    pad_forward(project_dir + "/index_label.pkl", project_dir + "/index_label_nn.pkl",
                summary_len=50, description_len=100)

    #将整理好的数据划分为训练集、验证集、测试集
    input_file = open(project_dir + "/index_label_nn.pkl", "rb")
    data = pickle.load(input_file)
    input_file.close()
    train_valid_data = data[:int(len(data) * 0.9)]
    train_valid_data = shuffle(train_valid_data, random_state=0)
    train_data = train_valid_data[:int(len(train_valid_data) * 0.9)]
    valid_data = train_valid_data[int(len(train_valid_data) * 0.9):]
    test_data = data[int(len(data) * 0.9):]
    pickle.dump(train_data, open(project_dir + "/train_nn.pkl", 'wb'))
    pickle.dump(valid_data, open(project_dir + "/valid_nn.pkl", 'wb'))
    pickle.dump(test_data, open(project_dir + "/test_nn.pkl", 'wb'))


In [9]:
def preprocess_all():

    all_dir = output_dir + '\\' + 'all'
    if not os.path.exists(all_dir):
        os.makedirs(all_dir)

    word2index = {}
    index_label = []
    index = 1  # 0 used for padding
    print('t1')

    for project_name in ["jackrabbit", "lucene", "httpclient"]:
        print('t2')
        input_file = open("dataset/" + project_name + ".pkl", "rb")
        info = pickle.load(input_file)
        input_file.close()

        key = []
        summary = []
        description = []
        priority = []
        for i in range(len(info)):
            key.append(info[i]['key'])
            summary.append(info[i]['fields']['summary'])
            description.append(info[i]['fields']['description'])
            priority.append(info[i]['fields']['priority']['id'])

        # 将summary、description分别分词、去停用词
        summary_tokenize = []
        description_tokenize = []
        for i in range(len(summary)):
            cur_summary = summary[i]
            cur_description = description[i]
            summary_words = list(word_tokenize(cur_summary))
            summary_words = [word.lower() for word in summary_words]
            summary_words = remove_stop_words(summary_words, stop_words_path)
            summary_words = stem_words(summary_words)
            try:
                description_words = list(word_tokenize(cur_description))
                description_words = [word.lower() for word in description_words]
                description_words = remove_stop_words(description_words, stop_words_path)
                description_words = stem_words(description_words)
            except Exception:
                description_words = []
            summary_tokenize.append(summary_words)
            description_tokenize.append(description_words)
        summary_tokenize = np.array(summary_tokenize)
        description_tokenize = np.array(description_tokenize)

        # 将经过分词、去停用词后的summary、description中的数字和“.”置换为“<NUM>” here error
        pattern1 = re.compile("^[0-9\.]+$")
        summary_processed = []
        for i in range(len(summary_tokenize)):
            cur_summary_tokenize = []
            for word in summary_tokenize[i]:
                if pattern1.match(word) is not None:
                    cur_summary_tokenize.append("<NUM>")
                    continue
                cur_summary_tokenize.append(word)
            summary_processed.append(cur_summary_tokenize)

        description_processed = []
        for i in range(len(description_tokenize)):
            cur_description_tokenize = []
            for word in description_tokenize[i]:
                if pattern1.match(word) is not None:
                    cur_description_tokenize.append("<NUM>")
                    continue
                cur_description_tokenize.append(word)
            description_processed.append(cur_description_tokenize)

        summary_processed = np.array(summary_processed)
        description_processed = np.array(description_processed)

        info = pd.read_csv("dataset/" + project_name + "_classification_vs_type.csv")
        label = list((info['CLASSIFIED'] == "BUG").astype(int))

        for i in range(len(summary_processed)):
            summary_index = []
            description_index = []

            for word in summary_processed[i]:
                if word not in word2index:
                    word2index[word] = index
                    summary_index.append(index)
                    index += 1
                else:
                    summary_index.append(word2index[word])

            for word in description_processed[i]:
                if word not in word2index:
                    word2index[word] = index
                    description_index.append(index)
                    index += 1
                else:
                    description_index.append(word2index[word])

            index_label.append([key[i], summary_index, description_index, priority[i], label[i]])

    # 将word2index、index_label写入文件
    pickle.dump(word2index, open(all_dir + "/word2index.pkl", 'wb'))
    pickle.dump(index_label, open(all_dir + "/index_label.pkl", 'wb'))

    # 对句向量进行截取或填充
    pad_forward(all_dir + "/index_label.pkl", all_dir + "/index_label_nn.pkl",
                summary_len=50, description_len=100)

    # 将整理好的数据划分为训练集、验证集、测试集
    input_file = open(all_dir + "/index_label_nn.pkl", "rb")
    data = pickle.load(input_file)
    input_file.close()

    info = []
    for project_name in ['jackrabbit', 'lucene', 'httpclient']:
        input_file = open("dataset/" + project_name + ".pkl", "rb")
        info += pickle.load(input_file)
        input_file.close()

    key_id = []
    for i in range(len(info)):
        key_id.append(info[i]['id'])
    key_id = np.array(key_id).astype(int)
    indicies = np.argsort(key_id)

    train_valid_indicies = indicies[:int(len(indicies) * 0.9)]
    test_indicies = indicies[int(len(indicies) * 0.9):]

    train_valid_data = [data[i] for i in train_valid_indicies]
    test_data = [data[i] for i in test_indicies]

    train_valid_data = shuffle(train_valid_data, random_state=0)
    train_data = train_valid_data[:int(len(train_valid_data) * 0.95)]
    valid_data = train_valid_data[int(len(train_valid_data) * 0.95):]

    pickle.dump(train_data, open(all_dir + "/train_nn.pkl", 'wb'))
    pickle.dump(valid_data, open(all_dir + "/valid_nn.pkl", 'wb'))
    pickle.dump(test_data, open(all_dir + "/test_nn.pkl", 'wb'))


In [12]:
if __name__ == "__main__":
#     print("preprocess jackrabbit")
#     preprocess_project("jackrabbit")
#     print("preprocess lucene")
#     preprocess_project("lucene")
#     print("preprocess httpclient")
#     preprocess_project("httpclient")
#     print("preprocess all")
#     preprocess_all()

#     inputs = ['i', 'me', 'apple', 'you', 'orange', 'yourself', '?', 'oranges']
#     outputs = remove_stop_words(inputs, stop_words_path)
#     outputs = stem_words(outputs)
#     print(outputs)

#     cur_summary = "HttpState.clearCookies() should be synchronized"
#     summary_words = list(word_tokenize(cur_summary))
#     print(summary_words)

    project_name = "httpclient"
    info = pd.read_csv("dataset/" + project_name + "_classification_vs_type.csv")
    label = list((info['CLASSIFIED'] == "BUG").astype(int))
    label = np.array(label)
    number_bug = 0
    number_non_bug = 0
    print(len(label))
    for i in range(len(label)):
        if label[i] == 1:
            number_bug += 1
        elif label[i] == 0:
            number_non_bug += 1
    
    print(number_bug)
    print(number_non_bug)

746
305
441
